## Подготовка данных и разведовательный анализ

In [9]:
# imports
import pandas as pd
from functools import reduce

#### Константы
- пути к данным
- признаки, описывающие данные
- типы "столбцов", или "подпризнаков"
- ID ТС

In [10]:
DATA_PATH = 'data/vehicle_dataset_public/'
DATA_PROC_PATH = 'data/vehicle_proccessed/'

TAGS_FEATURES = ['fuelLevel', 'ingection', 'speedAndHeight', 'tachometer']
TAGS_TARGET = ['refueling2']
TAGS = TAGS_FEATURES + TAGS_TARGET

TIME_COLUMNS = ['DTIME', 'STARTDATE', 'ENDDATE']
FLOAT_COLUMNS = ['BEVALUE', 'SPEED', 'HEIGHT']

VEHICLE_IDS = [1, 3, 5, 19, 28]

In [11]:
# Словарь, состоящий из dataframe'ов для каждого признака
def dict_df_vehicle_i(i):
    dict_df_vehicle = {}
    for tag in TAGS:
        df = pd.read_csv(DATA_PATH +
                         f'vehicle{i}_{tag}_public.csv', sep=';')
        for col in FLOAT_COLUMNS:
            if col in df.columns and df[col].dtype == 'object':
                df[col] = df[col].str.replace(',','.').astype(float)
        df.rename(columns={'BEVALUE': tag}, inplace=True)

        for time_col in TIME_COLUMNS:
            if time_col in df.columns:
                df[time_col] = pd.to_datetime(df[time_col])

        dict_df_vehicle[tag] = df

    return dict_df_vehicle

In [12]:
# Объединение всех "исходных" признаков из словаря dataframe'ов
# в один dataframe
def df_summary(dict_df):
    df_to_merge = [dict_df[name] for name in TAGS_FEATURES]

    df_res = reduce(lambda left, right: pd.merge(left, right, how='outer', on='DTIME'),
                    df_to_merge)
    df_res.columns = df_res.columns.str.lower()

    return df_res

In [13]:
def combine_data():
    for i in VEHICLE_IDS:
        dict_df = dict_df_vehicle_i(i)

        df_sum = df_summary(dict_df)
        df_refuel = dict_df['refueling2']

        df = df_sum.copy()
        df['refuel'] = [1 if ((df_refuel['STARTDATE'] < date) & (date <= df_refuel['ENDDATE'])).any()
                        else 0 for date in df_sum['dtime']]

        name = f'vehicle{i}'
        df.to_csv(DATA_PROC_PATH + name + '.csv', index_label='i')

In [14]:
combine_data()

KeyboardInterrupt: 

In [ ]:
def df_for_all_vehicles():
    df_sum_list = []
    for i in VEHICLE_IDS:
        df_sum = pd.read_csv(DATA_PROC_PATH + f'vehicle{i}.csv')
        df_sum['TSID'] = str(i)

        df_sum_list.append(df_sum)

    df_vehicles_all = pd.concat(df_sum_list, ignore_index=True)

    return df_vehicles_all

In [ ]:
df_vehicles_all = df_for_all_vehicles()

df_vehicles_all


#### Составление pandas_profiling отчетов

In [15]:
import pandas as pd
import pandas_profiling

In [23]:
for i in VEHICLE_IDS:
    name = f'vehicle{i}'

    df_vehicle = pd.read_csv(DATA_PROC_PATH + name + '.csv')
    df_vehicle.dropna()

    report = pandas_profiling.ProfileReport(df_vehicle, explorative=True)
    report.to_file(DATA_PROC_PATH + 'reports/' + name +'.csv')



C:\Users\MikhailS\anaconda3\lib\site-packages\pandas_profiling\profile_report.py:262: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "
C:\Users\MikhailS\anaconda3\lib\site-packages\pandas_profiling\profile_report.py:262: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "
C:\Users\MikhailS\anaconda3\lib\site-packages\pandas_profiling\profile_report.py:262: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "
C:\Users\MikhailS\anaconda3\lib\site-packages\pandas_profiling\profile_report.py:262: UserWarning: Extension .cs